In [13]:
# Aiyagari Model adapted from George Hall code, http://people.brandeis.edu/~ghall/econ303/
# tested in Julia 1.11
# this code is part of chapter 6, "Heterogeneous Agents Models" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com
#using Pkg
#Pkg.add("Distributions")
using LinearAlgebra;
using SparseArrays;
using Distributions;
using Plots;

#  set parameter values
sigma  = 1.50;            # risk aversion              
beta   = 0.98;            # subjective discount factor 
prob   = [ .8 .2; .5 .5]; # prob(i,j) = probability (s(t+1)=sj | s(t) = si)
delta  = 0.97;            # 1 - depreciation
A      = 1.00;            # production technology
alpha  = 0.25;            # capital's share of income
theta  = 0.05;            # non-rental income if unemployed is theta*wage
Kstart = 10.0;            # initial value for aggregate capital stock
g      = 0.20;            # relaxation parameter

#   form capital grid
maxkap = 20;                      # maximum value of capital grid   
inckap = 0.025;                   # size of capital grid increments
nkap   = trunc(Int,maxkap/inckap+1);  # number of grid points: make it integer - Julia indexes must be integer

#global variables
decis  = zeros(nkap,2);
lambda = zeros(nkap,2);
probk  = zeros(nkap,1);

#   calculate aggregate labor supply
D = [0.0 0.0; 0.0 0.0];
ed,ev = eigen(prob,sortby = x -> -abs(x));


#   make a matrix from the eigenvalues
edm   = diagm(ed)
(emax,inmax) = findmax(edm)

D[inmax[1],inmax[2]] = emax;
pinf = ev*D*inv(ev);
pempl = pinf[inmax[1],inmax[2]];
N = 1.0*pempl + theta*(1-pempl);



In [14]:
liter   = 1;
maxiter = 50;
toler   = 0.001;
metric  = 10.0;
K = Kstart;
Kold= K;
wage=1.0;
rent=1.0;
println("ITERATING ON K");
println("");  
println("Iter    metric                       meanK                        Kold");

#  loop to find fixed point for agregate capital stock
  while  (metric[1] > toler) & (liter <= maxiter);

    
# calculate rental rate of capital and wage
   #
   wage = (1-alpha) * A * K^(alpha)   * N^(-alpha);
   rent = (alpha)   * A * K^(alpha-1) * N^(1-alpha);
   # 
   #  tabulate the utility function such that for zero or negative
   #  consumption utility remains a large negative number so that
   #  such values will never be chosen as utility maximizing      
   #
   util1=-10000*ones(nkap,nkap);  # utility when employed     
   util2=-10000*ones(nkap,nkap);  # utility when unemployed   
   for i=1:nkap;
         kap=(i-1)*inckap;
         for j=1:nkap; 
               kapp = (j-1)*inckap;
               cons1 = wage + (rent + delta)*kap - kapp; 
               if cons1 > .0;
                  util1[j,i]=(cons1)^(1-sigma)/(1-sigma);
               end;
               cons2 = theta*wage + (rent + delta)*kap - kapp;
               if cons2 > .0;
                  util2[j,i]=(cons2)^(1-sigma)/(1-sigma);
               end;
         end;
   end;
   #
   #  initialize some variables
   #
   v       = zeros(nkap,2);
   tdecis1 = zeros(nkap,1);
   tdecis2 = zeros(nkap,1);
   tdecis  = zeros(nkap,2)

   test    = 10;
   rs,cs = size(util1)
   r1=zeros(cs,cs);
   r2=zeros(cs,cs);
   indx=zeros(cs,1)
  
  while test != 0;
       for i=1:cs;
           r1[:,i]=util1[:,i]+beta*(prob[1,1]*v[:,1]+ prob[1,2]*v[:,2]);
           r2[:,i]=util2[:,i]+beta*(prob[2,1]*v[:,1]+ prob[2,2]*v[:,2]);
       end;

    (tv1,tdecis1)=findmax(r1,dims=1)
    (tv2,tdecis2)=findmax(r2,dims=1);

  for i=1:cs;
    tdecis[i,1]=tdecis1[i][1]
    tdecis[i,2]=tdecis2[i][1]
  end;
 
  tv=[tv1' tv2']
  test=maximum((tdecis-decis));
       copy!(v, tv);
       copy!(decis, tdecis);
  end;     

  decis=(decis.-1).*inckap;

#   form transition matrix
#   trans is the transition matrix from state at t (row)
#   to the state at t+1 (column) 
#   The eigenvector associated with the unit eigenvalue
#   of trans' is  the stationary distribution. 
 
   g2=spzeros(cs,cs);
   g1=spzeros(cs,cs);
   for i=1:cs
       g1[i,tdecis1[i][1]]=1;
       g2[i,tdecis2[i][1]]=1;
   end
  
   trans=[ prob[1,1]*g1 prob[1,2]*g1; prob[2,1]*g2 prob[2,2]*g2];
   trans=trans';
   probst = (1/(2*nkap))*ones(2*nkap,1);
   test=1;
   while test > 10.0^(-8);
      probst1 = trans*probst;
      test = maximum(abs.(probst1-probst));
      copy!(probst, probst1);
   end;
 
 #   vectorize the decision rule to be conformable with probst
 #   calculate new aggregate capital stock  meanK  

  kk=vec(decis);
  meanK=probst'*kk;

  #  calculate measure over (k,s) pairs
  #  lambda has same dimensions as decis
  lambda=reshape(probst, cs,2)


  #   calculate stationary distribution of k
  d1,v1=eigen(prob',sortby = x -> -abs(x))

  d1m  = diagm(d1);
  dmax,imax=findmax(diag(d1m));
  probst1=v1[:,imax];
  ss=sum(probst1);
  probst1=probst1./ss;
  probk=sum(lambda,dims=2);
 
  #   form metric and update K
  Kold= K;
  Knew= g*meanK[1] + (1-g)*Kold;
  metric = abs.((Kold.-meanK)./Kold);
  K = Knew;
  println(liter,"        ", metric[1],"           ",meanK[1],"            ",Kold);
  liter = liter+1;
end;

ITERATING ON K

Iter    metric                       meanK                        Kold
1        0.5499633921667572           4.500366078332428            10.0
2        0.4544142973494472           4.855752699010765            8.900073215666486
3        0.3404741462790561           5.336361597447647            8.091209112335342
4        0.21417551289286063           5.9253049236885325            7.540239609357803
5        0.10094935807960213           6.488675647864649            7.21725267222395
6        0.033215148107074696           6.836655109672292            7.07153726735209
7        0.009285198741766663           6.959336392381948            7.024560835816131
8        0.0006063274285822773           7.015767221563982            7.011515947129295


In [15]:
#print  results
println("PARAMETER VALUES");
println("");
println("sigma      beta      delta       A       alpha      theta"); 
println(sigma,"        ",beta, "        ",delta,"     ", A, "   ",alpha,"   ",theta);
println(""); 
println("EQUILIBRIUM RESULTS ");
println();
println("K               N              wage             rent");
println(round(Kold; digits=2),"          ",round(N; digits=2), "             ",round(wage; digits=2),"            ", round(rent; digits=2));

PARAMETER VALUES

sigma      beta      delta       A       alpha      theta
1.5        0.98        0.97     1.0   0.25   0.05

EQUILIBRIUM RESULTS 

K               N              wage             rent
7.01          0.73             1.32            0.05


In [16]:
#  function markov(T,n,s0,V);
#  chain generates a simulation from a Markov chain of dimension
#  the size of T
#  T is transition matrix
#  n is number of periods to simulate
#  s0 is initial state
#  V is the quantity corresponding to each state
#  state is a matrix recording the number of the realized state at time t

function markov(T,n,s0,V);
    
r,c   = size(T);
v1,v2 = size(V);

#rand('uniform');
#using Distributions
X=rand(Uniform(0,1), n-1,1)

state=zeros(2,99);
chain=[];
s=zeros(r,1);
s[s0]=1
cum=T*triu(ones(size(T)));
ppi  =[];

state[:,1]=s

for k=1:length(X);
#k=1
  state[:,k]=s;
  ppi =[0 s'*cum];
  ss1= convert(Array{Float64},((X[k].<=ppi[2:r+1])))
  ss2= convert(Array{Float64},(X[k].>ppi[1:r]))
  s=(ss1.*ss2) 
  s=reshape(s,2,1)
end    

chain=V*state
return state,chain    
end

markov (generic function with 1 method)

In [17]:
#    simulate life histories of the agent
#rand('uniform');


println("SIMULATING LIFE HISTORY");
k = Kold;               # initial level of capital 
n = 100;                # number of periods to simulate
s0 = 1;                 # initial state 
hist = zeros(n-1,2);
cons = zeros(n-1,1);
invest = zeros(n-1,1);

grid  = collect(0:inckap:maxkap);
r,c   = size(prob);
T     = prob;
V     = collect(1:r)';

state, chain= markov(prob,n,s0,V)

chain=convert(Array{Int64,2},chain);
state=convert(Array{Int64,2},state);

SIMULATING LIFE HISTORY


In [18]:

for i = 1:n-1;
    hist[i,:] = [ k chain[i] ];
    I1 = trunc(Int,k/inckap) ;
    I2 = trunc(Int,k/inckap) + 1;
    if I1 == 0;
       I1=1;
       println("N.B.  I1 = 0");
    end;
    if I2 > nkap;
       I2 = nkap;
       println("N.B.  I2 > nkap");
    end;
    weight = (grid[I2,1] - k)/inckap; 
    kprime = weight*(decis[I1,chain[i]]) +  (1-weight)*(decis[I2,chain[i]]);
    if chain[i] == 1;
       cons[i] = wage + (rent + delta)*k - kprime;
    elseif chain[i] == 2;
       cons[i] = wage*theta + (rent + delta)*k - kprime;
    else;
      println("something is wrong with chain");
      chain
    end;
    k = kprime;
    invest[i] = kprime;
end;


In [19]:
#plots
#Income distribution
income =  [rent*grid.+ wage  rent*grid.+wage*theta]   ; 
#sort income
income =  [rent*grid.+ wage  rent*grid.+wage*theta]  
pinc = sort(vec(income));
index = sortperm(vec(income));
index=trunc.(Int,index);
plambda = vec(lambda);

In [20]:
#using Plots
#import PlotlyBase
using Pkg
#Pkg.add("PlotlyBase")
#using Plots
plotly() 
plot(pinc,plambda[index], linewidth=1,title="Income Distribution", ylabel="% of agents", label="income level")

In [21]:
#plots
#Capital distribution
#using Plots
plotly() 
plot(grid,probk, linewidth=1,title="Capital Distribution", ylabel="% of agents", label="capital")
